In [3]:
import os

In [1]:
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\ankur\DL end 2 end project\kidney-Disease-Classification-Using-MLflow-DVC\research


In [4]:
import os
print(os.path.exists("config/config.yaml"))
print(os.path.exists("params.yaml"))


True
True


In [4]:
%pwd

'C:\\ankur\\DL end 2 end project\\kidney-Disease-Classification-Using-MLflow-DVC'

In [3]:
os.chdir("../")

In [6]:
%pwd

'C:\\ankur\\DL end 2 end project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-11-02 19:02:21,092:INFO:common:yaml file: config\config.yaml loaded sucessfully]
[2025-11-02 19:02:21,114:INFO:common:yaml file: params.yaml loaded sucessfully]
[2025-11-02 19:02:21,116:INFO:common: created directories at: artifacts]
[2025-11-02 19:02:21,118:INFO:common: created directories at: artifacts\training]
Found 120 images belonging to 2 classes.
Found 480 images belonging to 2 classes.
Epoch 1/100
30/30 [==============================] - 76s 3s/step - loss: 9.8679 - accuracy: 0.5958 - val_loss: 16.7434 - val_accuracy: 0.3125
Epoch 2/100
30/30 [==============================] - 65s 2s/step - loss: 5.8319 - accuracy: 0.7292 - val_loss: 9.8879 - val_accuracy: 0.5714
Epoch 3/100
30/30 [==============================] - 66s 2s/step - loss: 1.9806 - accuracy: 0.8250 - val_loss: 6.9209 - val_accuracy: 0.3393
Epoch 4/100
30/30 [==============================] - 65s 2s/step - loss: 2.9546 - accuracy: 0.7958 - val_loss: 4.5101 - val_accuracy: 0.5714
Epoch 5/100
30/30 [============

In [13]:
import os

print("Current working directory:", os.getcwd())
print("Files in current directory:", os.listdir())

if os.path.exists("config"):
    print("Files inside 'config' folder:", os.listdir("config"))
else:
    print("'config' folder does NOT exist")


Current working directory: C:\ankur\DL end 2 end project
Files in current directory: ['kidney-Disease-Classification-Using-MLflow-DVC', 'logs']
'config' folder does NOT exist


In [46]:
import os
print(os.getcwd())


c:\
